# OpenAI Agents SDK - 프로덕션 에이전트

이 튜토리얼은 OpenAI Agents SDK를 사용하여 프로덕션 수준의 AI 에이전트를 구축하는 방법을 다룬다. 세션 관리, 메모리 관리, 에러 핸들링, 모니터링, 토큰 추적 등 엔터프라이즈급 대화형 AI 구축에 필요한 모든 요소를 종합적으로 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 프로덕션 에이전트 구축

이 챕터에서는 실제 프로덕션 환경에서 사용할 수 있는 완전한 기능을 갖춘 에이전트를 구축한다. 지금까지 학습한 모든 패턴을 통합하여 엔터프라이즈급 대화형 AI 시스템을 만든다.

### 학습 내용

- 환경 설정 및 API 키 검증
- 프로덕션 로깅 설정
- 세션 컨텍스트 관리
- 도구 통합
- 세션별 메모리 관리
- 재시도 로직과 지수 백오프
- 토큰 사용량 추적 및 비용 모니터링
- 성능 모니터링 및 통계
- 세션 내보내기 및 분석

### Step 1: 환경 설정 및 검증

프로덕션 애플리케이션에서는 항상 환경 변수를 검증해야 한다. API 키가 없거나 형식이 잘못된 경우 조기에 실패하여 런타임 오류를 방지한다. 이는 배포 파이프라인에서 문제를 빠르게 발견할 수 있게 해주는 중요한 보안 관행이다.

In [1]:
import asyncio
import os
import json
import time
from dataclasses import dataclass, field
from typing import Dict, List, Any, Optional
from datetime import datetime
from dotenv import load_dotenv
import logging

from agents import Agent, Runner, ModelSettings, RunContextWrapper, function_tool

# 환경 변수 로드 및 검증
load_dotenv()

MODEL = "gpt-4o-mini"

### Step 2: 프로덕션 로깅 설정

프로덕션 애플리케이션에서는 구조화된 로깅이 필수적이다. 로깅을 통해 디버깅, 모니터링, 감사를 수행할 수 있다. 여기서는 콘솔 출력을 설정하지만, 실제 프로덕션 환경에서는 파일 로깅이나 클라우드 로깅 서비스(CloudWatch, Stackdriver 등)를 추가해야 한다.

In [2]:
# 프로덕션 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # 콘솔 출력
        # 프로덕션 환경에서는 추가: logging.FileHandler('agent.log')
    ]
)
logger = logging.getLogger(__name__)

logger.info("로깅 시스템 초기화 완료")

2025-12-07 23:31:37,428 - __main__ - INFO - 로깅 시스템 초기화 완료


### Step 3: 세션 컨텍스트 관리

세션 컨텍스트는 도구와 콜백에 전달되는 애플리케이션 데이터를 담는다. 사용자 정보, 세션 메타데이터, 데이터베이스 연결, API 클라이언트 등 에이전트가 작업 수행 시 필요한 모든 정보를 포함할 수 있다. `to_dict()` 메서드는 컨텍스트를 직렬화하여 로깅이나 저장에 활용할 수 있게 한다.

In [3]:
@dataclass
class SessionContext:
    """
    도구와 콜백에 전달되는 컨텍스트 객체이다.
    
    프로덕션 팁: 도구가 접근해야 하는 데이터를 여기에 저장한다:
    - 사용자 정보 (ID, 이름, 설정)
    - 데이터베이스 연결
    - API 클라이언트
    - 구성 설정
    """
    user_id: str
    session_id: str
    user_name: str = ""
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: datetime = field(default_factory=datetime.now)
    
    def to_dict(self) -> dict:
        """컨텍스트를 직렬화한다. 로깅 및 저장에 유용하다."""
        return {
            "user_id": self.user_id,
            "session_id": self.session_id,
            "user_name": self.user_name,
            "metadata": self.metadata,
            "created_at": self.created_at.isoformat()
        }

print("SessionContext 클래스가 정의되었다.")

SessionContext 클래스가 정의되었다.


### Step 4: 도구 정의 - 에이전트가 호출할 수 있는 함수

도구는 에이전트가 외부 시스템과 상호작용하는 방법이다. 데이터베이스 쿼리, API 호출, 파일 작업, 사용자 데이터 조회 등을 수행할 수 있다. 각 도구는 `RunContextWrapper`를 통해 세션 컨텍스트에 접근하고, 작업 결과를 문자열로 반환한다. 모든 도구 호출은 로깅하여 추적 가능하게 한다.

In [4]:
@function_tool
async def get_user_info(wrapper: RunContextWrapper[SessionContext]) -> str:
    """
    사용자 컨텍스트에 접근하는 도구 예제이다.
    
    프로덕션 팁: 도구는 에이전트가 외부 시스템과 상호작용하는 방법이다:
    - 데이터베이스 쿼리
    - API 호출
    - 파일 작업
    - 사용자 데이터 조회
    """
    ctx = wrapper.context
    logger.info(f"도구 호출: get_user_info - 사용자 {ctx.user_id}")
    return f"사용자: {ctx.user_name or '알 수 없음'} (ID: {ctx.user_id})"

@function_tool  
async def log_interaction(wrapper: RunContextWrapper[SessionContext]) -> str:
    """
    로깅 및 분석을 위한 도구 예제이다.
    
    프로덕션 팁: 도구를 사용하여 로깅, 이메일 전송,
    데이터베이스 업데이트 등의 부작용을 수행한다.
    """
    ctx = wrapper.context
    logger.info(f"상호작용 기록: 사용자 {ctx.user_id}, 세션 {ctx.session_id}")
    return "상호작용이 성공적으로 기록되었습니다."

print("도구 함수가 정의되었다: get_user_info, log_interaction")

도구 함수가 정의되었다: get_user_info, log_interaction


### Step 5: 프로덕션 에이전트 클래스

프로덕션 에이전트 클래스는 엔터프라이즈 모범 사례를 구현한다. 주요 기능으로는 세션별 메모리 관리, 재시도 로직을 포함한 에러 핸들링, 토큰 사용량 추적, 성능 모니터링, 세션 관리, 포괄적인 로깅이 있다. 모델 설정은 항상 명시적으로 구성하며, 일관된 비즈니스 로직에는 낮은 temperature(0.1-0.3)를, 창의적 작업에는 높은 temperature(0.7-0.9)를 사용한다.

In [5]:
class ProductionAgent:
    """
    엔터프라이즈 모범 사례를 구현한 프로덕션 에이전트이다.
    
    주요 프로덕션 기능:
    - 세션별 메모리 관리
    - 재시도 로직을 포함한 에러 핸들링
    - 토큰 사용량 추적
    - 성능 모니터링
    - 세션 관리
    - 포괄적인 로깅
    """
    
    def __init__(self, name: str, instructions: str, **kwargs):
        """
        프로덕션 에이전트를 초기화한다.
        
        프로덕션 팁: 모델 설정을 항상 명시적으로 구성한다.
        일관된 비즈니스 로직에는 낮은 temperature(0.1-0.3),
        창의적 작업에는 높은 temperature(0.7-0.9)를 사용한다.
        """
        self.agent = Agent[SessionContext](
            name=name,
            instructions=instructions,
            model_settings=ModelSettings(
                temperature=kwargs.get('temperature', 0.3),  # 일관된 응답
                max_tokens=kwargs.get('max_tokens', 1000)     # 응답 길이 제어
            ),
            tools=[get_user_info, log_interaction],
            **{k: v for k, v in kwargs.items() if k not in ['temperature', 'max_tokens']}
        )
        
        # 메모리 관리: 세션별 대화 저장
        # 프로덕션 팁: 실제 앱에서는 Redis/Database에 저장하여 영속성 확보
        self.conversations: Dict[str, List[dict]] = {}
        
        # 모니터링: 프로덕션 관찰성을 위한 메트릭 추적
        self.total_tokens_used = 0
        self.session_count = 0
        self.error_count = 0
        
        logger.info(f"ProductionAgent '{name}' 초기화 완료")

### Step 5.1: 메인 채팅 메서드

`chat` 메서드는 프로덕션급 에러 핸들링과 모니터링을 포함한 핵심 대화 로직을 구현한다. 주요 프로덕션 원칙으로는 입력 검증, 세션 관리, 지수 백오프를 포함한 재시도 로직, 포괄적인 에러 핸들링, 성능 모니터링, 상세 로깅, 우아한 성능 저하가 있다.

In [6]:
# ProductionAgent 클래스에 chat 메서드 추가
async def chat(
    self, 
    message: str, 
    context: SessionContext,
    max_retries: int = 3
) -> dict:
    """
    프로덕션급 에러 핸들링과 모니터링을 포함한 메인 채팅 메서드이다.
    
    구현된 프로덕션 원칙:
    1. 입력 검증
    2. 세션 관리
    3. 지수 백오프를 포함한 재시도 로직
    4. 포괄적인 에러 핸들링
    5. 성능 모니터링
    6. 상세 로깅
    7. 우아한 성능 저하
    """
    session_id = context.session_id
    
    # 검증: 프로덕션에서는 항상 입력을 검증한다
    if not message or not message.strip():
        return {
            "response": "메시지를 입력해주세요.",
            "success": False,
            "error": "빈 메시지"
        }
    
    # 세션 관리: 새 세션 초기화
    if session_id not in self.conversations:
        self.conversations[session_id] = []
        self.session_count += 1
        logger.info(f"새 세션 시작: {session_id}, 사용자 {context.user_id}")
    
    # 메모리 관리: 기록과 함께 대화 준비
    conversation_history = self.conversations[session_id].copy()
    conversation_history.append({"role": "user", "content": message.strip()})
    
    # 재시도 로직: 프로덕션 안정성을 위해 필수
    for attempt in range(max_retries):
        try:
            logger.info(
                f"메시지 처리 중: {context.user_name} "
                f"(세션: {session_id}, 시도: {attempt + 1}/{max_retries})"
            )
            
            # 성능 모니터링: 응답 시간 추적
            start_time = time.time()
            
            # 핵심 실행: 메모리 + 컨텍스트와 함께 에이전트 실행
            result = await Runner.run(
                starting_agent=self.agent,
                input=conversation_history,    # 대화 메모리
                context=context,               # 도구용 사용자/앱 컨텍스트
                max_turns=5                    # 무한 루프 방지
            )
            
            response_time = time.time() - start_time
            
            # 메모리 업데이트: 전체 대화 상태 저장
            updated_history = result.to_input_list()
            self.conversations[session_id] = updated_history
            
            # 토큰 추적: API 사용량 및 비용 모니터링
            tokens_used = sum(r.usage.total_tokens for r in result.raw_responses)
            self.total_tokens_used += tokens_used
            
            # 성공 로깅: 성공한 작업 추적
            logger.info(
                f"메시지 처리 성공. "
                f"토큰: {tokens_used}, 시간: {response_time:.2f}초, "
                f"대화 길이: {len(updated_history)}"
            )
            
            # 구조화된 응답: 포괄적인 결과 반환
            return {
                "response": result.final_output,
                "session_id": session_id,
                "tokens_used": tokens_used,
                "response_time": response_time,
                "conversation_length": len(updated_history),
                "success": True,
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            # 에러 핸들링: 실패 로깅 및 추적
            self.error_count += 1
            logger.warning(f"시도 {attempt + 1} 실패: {str(e)}")
            
            # 최종 시도: 우아한 성능 저하 제공
            if attempt == max_retries - 1:
                error_msg = "기술적인 문제가 발생했습니다. 다시 시도해 주세요."
                logger.error(
                    f"세션 {session_id}의 모든 재시도 실패. "
                    f"오류: {str(e)}"
                )
                return {
                    "response": error_msg,
                    "session_id": session_id,
                    "error": str(e),
                    "success": False,
                    "timestamp": datetime.now().isoformat()
                }
            
            # 지수 백오프: 재시도 전 대기
            await asyncio.sleep(2 ** attempt)
                
    return {
        "success": False, 
        "error": "최대 재시도 횟수 초과",
        "timestamp": datetime.now().isoformat()
    }

# 메서드를 클래스에 바인딩
ProductionAgent.chat = chat
print("chat 메서드가 ProductionAgent 클래스에 추가되었다.")

chat 메서드가 ProductionAgent 클래스에 추가되었다.


### Step 5.2: 유틸리티 메서드

프로덕션 에이전트에는 대화 기록 조회, 세션 정리, 사용량 통계, 세션 내보내기 등의 유틸리티 메서드가 필요하다. 이러한 메서드들은 디버깅, 분석, 규정 준수, 메모리 관리에 활용된다.

In [7]:
def get_conversation_history(self, session_id: str) -> List[dict]:
    """대화 기록을 반환한다. 디버깅 및 분석에 유용하다."""
    return self.conversations.get(session_id, [])

def clear_session(self, session_id: str) -> bool:
    """대화 기록을 삭제한다. 메모리 관리에 중요하다."""
    if session_id in self.conversations:
        del self.conversations[session_id]
        logger.info(f"세션 {session_id} 삭제됨")
        return True
    logger.warning(f"존재하지 않는 세션 삭제 시도: {session_id}")
    return False

def get_usage_stats(self) -> dict:
    """
    프로덕션 모니터링을 위한 종합 사용량 통계이다.
    
    프로덕션 팁: 이 메트릭을 모니터링 시스템
    (Prometheus, DataDog 등)에 노출하여 관찰성을 확보한다.
    """
    active_sessions = len(self.conversations)
    total_messages = sum(len(conv) for conv in self.conversations.values())
    
    return {
        "total_tokens_used": self.total_tokens_used,
        "estimated_cost_usd": self.total_tokens_used * 0.00002,  # 대략적 비용
        "active_sessions": active_sessions,
        "total_sessions": self.session_count,
        "error_count": self.error_count,
        "error_rate": self.error_count / max(self.session_count, 1),
        "total_messages": total_messages,
        "average_conversation_length": (
            total_messages / max(active_sessions, 1)
        ),
        "uptime": datetime.now().isoformat()
    }

def export_session(self, session_id: str) -> Optional[dict]:
    """
    분석, 규정 준수 또는 디버깅을 위해 세션을 내보낸다.
    
    프로덕션 팁: 규정 준수, 사용자 데이터 요청,
    프로덕션 이슈 디버깅에 필수적이다.
    """
    if session_id not in self.conversations:
        logger.warning(f"존재하지 않는 세션 내보내기 요청: {session_id}")
        return None
        
    session_data = {
        "session_id": session_id,
        "conversation": self.conversations[session_id],
        "message_count": len(self.conversations[session_id]),
        "exported_at": datetime.now().isoformat(),
        "export_version": "1.0"
    }
    
    logger.info(f"세션 {session_id} 내보내기 완료: {session_data['message_count']}개 메시지")
    return session_data

# 메서드들을 클래스에 바인딩
ProductionAgent.get_conversation_history = get_conversation_history
ProductionAgent.clear_session = clear_session
ProductionAgent.get_usage_stats = get_usage_stats
ProductionAgent.export_session = export_session

print("유틸리티 메서드가 ProductionAgent 클래스에 추가되었다.")

유틸리티 메서드가 ProductionAgent 클래스에 추가되었다.


### Step 6: 프로덕션 데모 및 테스트

구축한 프로덕션 에이전트의 모든 기능을 테스트한다. 다중 사용자 세션 관리, 상호작용 간 메모리 유지, 컨텍스트를 활용한 도구 사용, 에러 핸들링과 모니터링, 사용량 분석을 시연한다.

In [8]:
async def production_demo():
    """
    프로덕션 에이전트 기능을 종합적으로 시연하는 함수이다.
    
    시연 내용:
    - 다중 사용자 세션 관리
    - 상호작용 간 메모리 유지
    - 컨텍스트를 활용한 도구 사용
    - 에러 핸들링과 모니터링
    - 사용량 분석
    """
    
    print("프로덕션 에이전트 데모")
    print("=" * 50)
    
    # 프로덕션 에이전트 초기화
    agent = ProductionAgent(
        name="ProductionCustomerSupport",
        instructions="""당신은 전문 고객 지원 에이전트입니다.
        도움이 되고, 대화 컨텍스트를 기억하며, 적절할 때 도구를 사용하세요.
        항상 정중하고 전문적이어야 합니다.""",
        temperature=0.2  # 일관된 지원 응답을 위한 낮은 temperature
    )
    
    return agent

# 에이전트 초기화
agent = await production_demo()

2025-12-07 23:32:18,710 - __main__ - INFO - ProductionAgent 'ProductionCustomerSupport' 초기화 완료


프로덕션 에이전트 데모


### 데모 1: 고객 지원 세션

첫 번째 고객의 지원 세션을 시뮬레이션한다. 세션 컨텍스트에 사용자 정보와 메타데이터를 설정하고, 여러 턴의 대화를 수행하여 메모리가 유지되는지 확인한다.

In [9]:
print("\n데모 1: 고객 지원 세션")
print("-" * 30)

# 첫 번째 고객 컨텍스트 생성
customer_context = SessionContext(
    user_id="customer_001",
    session_id="support_session_123", 
    user_name="김철수",
    metadata={
        "account_type": "프리미엄", 
        "region": "서울",
        "support_tier": "우선"
    }
)

# 첫 번째 상호작용
result1 = await agent.chat(
    "안녕하세요, 프리미엄 기능에 접근하는 데 문제가 있습니다.",
    customer_context
)
print(f"고객: 안녕하세요, 프리미엄 기능에 접근하는 데 문제가 있습니다.")
print(f"에이전트: {result1['response']}")
print(f"사용된 토큰: {result1.get('tokens_used', 0)}")

2025-12-07 23:32:30,827 - __main__ - INFO - 새 세션 시작: support_session_123, 사용자 customer_001
2025-12-07 23:32:30,828 - __main__ - INFO - 메시지 처리 중: 김철수 (세션: support_session_123, 시도: 1/3)



데모 1: 고객 지원 세션
------------------------------


2025-12-07 23:32:34,776 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-07 23:32:34,835 - __main__ - INFO - 메시지 처리 성공. 토큰: 324, 시간: 4.00초, 대화 길이: 2


고객: 안녕하세요, 프리미엄 기능에 접근하는 데 문제가 있습니다.
에이전트: 안녕하세요! 프리미엄 기능에 접근하는 데 문제가 있으시군요. 불편을 드려 죄송합니다.  
문제를 더 잘 도와드리기 위해 아래 정보를 알려주실 수 있을까요?

1. 사용 중인 서비스나 앱 이름  
2. 프리미엄 기능에 접근하려고 할 때 나타나는 오류 메시지(있다면)  
3. 최근에 결제나 구독 변경 사항이 있었는지 여부  

이 정보를 알려주시면 빠르게 문제 해결을 도와드리겠습니다!
사용된 토큰: 324


2025-12-07 23:32:36,609 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-12-07 23:32:37,341 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-12-07 23:32:42,915 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-12-07 23:32:48,372 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-12-07 23:32:54,385 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In [10]:
# 후속 대화 - 메모리 테스트
result2 = await agent.chat(
    "제 문제가 뭐였죠? 제가 누군지 확인해 주시겠어요?",
    customer_context
)
print(f"\n고객: 제 문제가 뭐였죠? 제가 누군지 확인해 주시겠어요?")
print(f"에이전트: {result2['response']}")
print("\n[참고] 에이전트가 이전 대화 내용과 사용자 정보를 기억하고 있다.")

2025-12-07 23:32:34,838 - __main__ - INFO - 메시지 처리 중: 김철수 (세션: support_session_123, 시도: 1/3)
2025-12-07 23:32:36,404 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-07 23:32:36,408 - __main__ - INFO - 도구 호출: get_user_info - 사용자 customer_001
2025-12-07 23:32:39,224 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-07 23:32:39,228 - __main__ - INFO - 메시지 처리 성공. 토큰: 854, 시간: 4.39초, 대화 길이: 6



고객: 제 문제가 뭐였죠? 제가 누군지 확인해 주시겠어요?
에이전트: 고객님, 성함은 김철수님이시고, 고객 ID는 customer_001입니다.

문의해주신 문제는 "프리미엄 기능에 접근하는 데 문제가 있다"는 내용이었습니다.  
혹시 추가로 설명해주실 부분이나, 구체적으로 어떤 프리미엄 기능에서 문제가 발생하는지 말씀해주시면 더욱 정확하게 도와드릴 수 있습니다.  
불편을 드려 죄송하며, 빠르게 해결해드릴 수 있도록 최선을 다하겠습니다!

[참고] 에이전트가 이전 대화 내용과 사용자 정보를 기억하고 있다.


### 데모 2: 다른 고객, 별도 세션

두 번째 고객의 세션을 시작한다. 세션이 격리되어 있어 첫 번째 고객의 정보에 접근할 수 없음을 확인한다. 이는 프로덕션 환경에서 데이터 보안과 프라이버시를 위해 중요하다.

In [11]:
print("\n\n데모 2: 다른 고객 세션")
print("-" * 30)

# 두 번째 고객 컨텍스트 생성
customer2_context = SessionContext(
    user_id="customer_002",
    session_id="support_session_456",
    user_name="이영희",
    metadata={"account_type": "기본", "region": "부산"}
)

result3 = await agent.chat(
    "김철수 고객의 문제가 뭐였는지 아시나요?",
    customer2_context
)
print(f"이영희: 김철수 고객의 문제가 뭐였는지 아시나요?")
print(f"에이전트: {result3['response']}")
print("\n[확인] 에이전트는 김철수의 정보를 모른다 - 세션이 격리되어 있다!")

2025-12-07 23:32:44,762 - __main__ - INFO - 새 세션 시작: support_session_456, 사용자 customer_002
2025-12-07 23:32:44,763 - __main__ - INFO - 메시지 처리 중: 이영희 (세션: support_session_456, 시도: 1/3)




데모 2: 다른 고객 세션
------------------------------


2025-12-07 23:32:46,017 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-07 23:32:46,021 - __main__ - INFO - 도구 호출: get_user_info - 사용자 customer_002
2025-12-07 23:32:48,596 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-07 23:32:48,600 - __main__ - INFO - 메시지 처리 성공. 토큰: 530, 시간: 3.84초, 대화 길이: 4


이영희: 김철수 고객의 문제가 뭐였는지 아시나요?
에이전트: 현재 고객님의 이름은 이영희로 확인됩니다. 김철수 고객님의 정보나 문제에 대해서는 현재 접근할 수 없습니다. 김철수 고객님의 문제에 대해 더 자세히 알고 싶으시면, 관련 정보를 제공해 주시거나 구체적으로 어떤 점을 확인하고 싶으신지 말씀해 주세요.

[확인] 에이전트는 김철수의 정보를 모른다 - 세션이 격리되어 있다!


### 데모 3: 프로덕션 모니터링

프로덕션 환경에서 필수적인 모니터링 기능을 시연한다. 사용량 통계를 확인하고, 세션 데이터를 내보내어 분석이나 규정 준수 목적으로 활용할 수 있다.

In [12]:
print("\n\n데모 3: 프로덕션 모니터링")
print("-" * 30)

# 사용량 통계 표시
stats = agent.get_usage_stats()
print("프로덕션 메트릭:")
print(json.dumps(stats, indent=2, ensure_ascii=False))



데모 3: 프로덕션 모니터링
------------------------------
프로덕션 메트릭:
{
  "total_tokens_used": 1708,
  "estimated_cost_usd": 0.03416,
  "active_sessions": 2,
  "total_sessions": 2,
  "error_count": 0,
  "error_rate": 0.0,
  "total_messages": 10,
  "average_conversation_length": 5.0,
  "uptime": "2025-12-07T23:32:49.247283"
}


In [13]:
# 분석을 위한 세션 내보내기
session_export = agent.export_session("support_session_123")
if session_export:
    print(f"\n세션 내보내기: {session_export['message_count']}개 메시지 내보냄")
    print(f"세션 ID: {session_export['session_id']}")
    print(f"내보내기 시간: {session_export['exported_at']}")

2025-12-07 23:32:51,225 - __main__ - INFO - 세션 support_session_123 내보내기 완료: 6개 메시지



세션 내보내기: 6개 메시지 내보냄
세션 ID: support_session_123
내보내기 시간: 2025-12-07T23:32:51.225258


### 데모 4: 에러 핸들링 테스트

입력 검증과 에러 핸들링이 올바르게 작동하는지 테스트한다. 빈 메시지를 전송하여 검증 로직이 적절한 응답을 반환하는지 확인한다.

In [14]:
print("\n\n데모 4: 에러 핸들링 테스트")
print("-" * 30)

# 빈 메시지로 테스트
error_result = await agent.chat("", customer_context)
print(f"빈 메시지 결과: {error_result}")



데모 4: 에러 핸들링 테스트
------------------------------
빈 메시지 결과: {'response': '메시지를 입력해주세요.', 'success': False, 'error': '빈 메시지'}


### 데모 5: 대화 기록 분석

저장된 대화 기록을 분석하여 전체 대화 흐름을 확인한다. 이는 품질 관리, 에이전트 성능 분석, 사용자 경험 개선에 활용할 수 있다.

In [15]:
print("\n\n데모 5: 대화 기록 분석")
print("-" * 30)

# 첫 번째 고객의 대화 기록 조회
history = agent.get_conversation_history("support_session_123")
print(f"김철수 고객 세션의 대화 기록: {len(history)}개 항목")

for i, item in enumerate(history):
    if isinstance(item, dict):
        role = item.get('role', 'unknown')
        content = str(item.get('content', ''))[:80]
        print(f"  {i+1}. [{role}] {content}{'...' if len(str(item.get('content', ''))) > 80 else ''}")



데모 5: 대화 기록 분석
------------------------------
김철수 고객 세션의 대화 기록: 6개 항목
  1. [user] 안녕하세요, 프리미엄 기능에 접근하는 데 문제가 있습니다.
  2. [assistant] [{'annotations': [], 'text': '안녕하세요! 프리미엄 기능에 접근하는 데 문제가 있으시군요. 불편을 드려 죄송합니다.  \...
  3. [user] 제 문제가 뭐였죠? 제가 누군지 확인해 주시겠어요?
  4. [unknown] 
  5. [unknown] 
  6. [assistant] [{'annotations': [], 'text': '고객님, 성함은 김철수님이시고, 고객 ID는 customer_001입니다.\n\n문의해주신...


### 데모 6: 세션 정리

메모리 관리를 위해 세션을 정리하는 방법을 시연한다. 세션 종료 시 또는 오래된 세션을 정리할 때 사용한다.

In [16]:
print("\n\n데모 6: 세션 정리")
print("-" * 30)

# 정리 전 통계
print(f"정리 전 활성 세션 수: {agent.get_usage_stats()['active_sessions']}")

# 세션 정리
agent.clear_session("support_session_456")

# 정리 후 통계
print(f"정리 후 활성 세션 수: {agent.get_usage_stats()['active_sessions']}")

2025-12-07 23:32:56,941 - __main__ - INFO - 세션 support_session_456 삭제됨




데모 6: 세션 정리
------------------------------
정리 전 활성 세션 수: 2
정리 후 활성 세션 수: 1


### 프로덕션 기능 요약

이 튜토리얼에서 시연한 프로덕션 기능들을 정리한다.

세션 간 메모리 관리

사용자 컨텍스트 및 도구 통합

에러 핸들링 및 검증

토큰 사용량 추적
성능 모니터링

세션 격리

포괄적인 로깅

데이터 내보내기 기능

세션 정리 및 메모리 관리

---

## 마무리

이 튜토리얼에서는 프로덕션 수준의 AI 에이전트를 구축하는 방법을 종합적으로 다루었다. 다음과 같은 내용을 학습하였다:

1. **환경 설정**: API 키 검증 및 프로덕션 로깅 설정
2. **세션 컨텍스트**: 사용자 정보와 메타데이터를 담는 컨텍스트 객체 설계
3. **도구 통합**: 컨텍스트에 접근하는 함수 도구 정의
4. **프로덕션 에이전트 클래스**: 메모리 관리, 재시도 로직, 모니터링을 포함한 완전한 에이전트 구현
5. **에러 핸들링**: 입력 검증, 지수 백오프, 우아한 성능 저하
6. **모니터링**: 토큰 추적, 사용량 통계, 세션 내보내기
7. **세션 관리**: 다중 사용자 지원, 세션 격리, 메모리 정리

이 패턴들을 적용하면 안정적이고 확장 가능하며 유지보수가 용이한 엔터프라이즈급 대화형 AI 시스템을 구축할 수 있다.